# CPA on Firmware Implementation of Magma (GOST)

## Magma Trace Capture

In [ ]:
SCOPETYPE = 'CWNANO' # options: OPENADC, CWNANO  
PLATFORM = 'CWNANO' # options: CWLITEXMEGA/CW308_XMEGA, CWLITEARM/CW308_STM32F3, CWNANO 
CRYPTO_TARGET='MAGMA'
SS_VER='SS_VER_1_1'

In [ ]:
%run "../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
scope.adc.samples = 50000 #options: 96000 for CW-PRO (CW1200), 24400 for CW-Lite, 131070 for CW-Husky

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../../firmware/mcu/simpleserial-gost
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3

In [ ]:
cw.program_target(scope, prog, "../../firmware/mcu/simpleserial-gost/simpleserial-gost-{}.hex".format(PLATFORM))

In [ ]:
from tqdm.notebook import trange
import numpy as np
import time
from os import urandom

trace_array = []
textin_array = []

text = urandom(16)

N = 350
if PLATFORM=="CWNANO": #increase nano reliability
    N = 100
for i in trange(N, desc='Capturing traces'):
    scope.arm()
    
    target.simpleserial_write('p', text)
    
    ret = scope.capture()
    if ret:
        print("Target timed out!")
        continue
    
    response = target.simpleserial_read('r', 16)
    
    trace_array.append(scope.get_last_trace())
    textin_array.append(text)
    
    text = urandom(16)
    
trace_array = np.array(trace_array)

In [ ]:
scope.dis()
target.dis()

In [ ]:
assert len(trace_array) == N
print("✔️ OK to continue!")

Again, let's quickly plot a trace to make sure everything looks as expected:

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt

plt.figure()
plt.plot(trace_array[0], 'r')
plt.plot(trace_array[N//2], 'g')
plt.plot(trace_array[N-1], 'b')
plt.show()

## Magma Model and Hamming Weight

In [ ]:
#TODO: Add functions for operations used in Magma cipher

HW = [bin(n).count("1") for n in range(0, 256)]

## Developing our Correlation Algorithm 

We'll be testing how good our guess is using a measurement called the Pearson correlation coefficient, which measures the linear correlation between two datasets. 

The actual algorithm is as follows for datasets $X$ and $Y$ of length $N$, with means of $\bar{X}$ and $\bar{Y}$, respectively:

$$r = \frac{cov(X, Y)}{\sigma_X \sigma_Y}$$

$cov(X, Y)$ is the covariance of `X` and `Y` and can be calculated as follows:

$$cov(X, Y) = \sum_{n=1}^{N}[(Y_n - \bar{Y})(X_n - \bar{X})]$$

$\sigma_X$ and $\sigma_Y$ are the standard deviation of the two datasets. This value can be calculated with the following equation:

$$\sigma_X = \sqrt{\sum_{n=1}^{N}(X_n - \bar{X})^2}$$

As you can see, the calulation is actually broken down pretty nicely into some smaller chunks that we can implement with some simple functions.

In [ ]:
def mean(X):
    return np.sum(X, axis=0)/len(X)

def std_dev(X, X_bar):
    return np.sqrt(np.sum((X-X_bar)**2, axis=0))

def cov(X, X_bar, Y, Y_bar):
    return np.sum((X-X_bar)*(Y-Y_bar), axis=0)

## Correlation Attack Implementaiton

In [ ]:
t_bar = np.sum(trace_array, axis=0)/len(trace_array)
o_t = np.sqrt(np.sum((trace_array - t_bar)**2, axis=0))

cparefs = [0] * 32 #put your key byte guess correlations here
bestguess = [0] * 32 #put your key byte guesses here

#TODO: Implement CPA attack on the Magma cipher, use Kuznyechick_CPA as a reference

With one final check to make sure you've got the correct key:

In [ ]:
key = [0x6c,0xec,0xc6,0x7f,0x28,0x7d,0x08,0x3d,
       0xeb,0x87,0x66,0xf0,0x73,0x8b,0x36,0xcf,
       0x16,0x4e,0xd9,0xb2,0x46,0x95,0x10,0x90,
       0x86,0x9d,0x08,0x28,0x5d,0x2e,0x19,0x3b]

for bnum in range(32):
    assert bestguess[bnum] == key[bnum], \
    "Byte {} failed, expected {:02X} got {:02X}".format(bnum, key[bnum], bestguess[bnum])
print("✔️ OK to continue!")